In [1]:
## Combining all the files labelled by the students into 1 file
import pandas as pd
import os

video_source = "C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Data/parkinsons_videos/labeled_data/"
output_files = "C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Data/parkinsons_videos/all_labeled_combined_file/"

file_data = pd.DataFrame()
for file_list1 in os.listdir(video_source):
    #print(file_list1)
    for files in os.listdir(video_source + file_list1 + "/"):
        #print(files)
        if not files.endswith("yaml"):
            for sub_files in os.listdir(video_source + file_list1 + "/" + files + "/"):
                if sub_files.endswith(".csv"):
                    #print(sub_files)
                    temp = pd.read_csv(video_source + file_list1 + "/" + files + "/" + sub_files, skiprows = 2)
                    temp = temp.iloc[:,0:5]
                    temp.columns = ["name", "finger_x","finger_y", "thumb_x","thumb_y"]
            file_data = pd.concat([file_data,temp], ignore_index = True)


In [2]:
## somehow full row dups are coming in. So remove them
file_data = file_data.drop_duplicates()

In [3]:
len(file_data)
file_data.to_csv("C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Data/parkinsons_videos/all_labeled_combined_file/combined_labels.csv")
print("the files are combined. Filename column is changed manually in csv outside of python - splitting name field")

the files are combined. Filename column is changed manually in csv outside of python - splitting name field


In [4]:
# Loading the updated single file 
print("loading the updated file")
labels_file = pd.read_csv("C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Data/parkinsons_videos/all_labeled_combined_file/combined_labels_v1_updated_filenames.csv")
labels_file.head()

loading the updated file


,video_name,seq,finger_x,finger_y,thumb_x,thumb_y
0,OC01_R_10s_lowfps,img000.png,527.435088,643.753157,568.125216,694.943964
1,OC01_R_10s_lowfps,img001.png,535.072768,643.354545,567.283288,691.011391
2,OC01_R_10s_lowfps,img002.png,533.811996,637.657582,567.946800,693.272491
3,OC01_R_10s_lowfps,img003.png,527.324229,636.657715,567.484718,691.878388
4,OC01_R_10s_lowfps,img004.png,528.169044,636.975038,567.431449,691.435793


In [5]:
# file_data.head()
# kk.loc[kk["name"].str.contains("YC101_R_10s_lowfps")]

In [6]:
labels_file["video_name"] = labels_file["video_name"] + ".mp4"
labels_file.head()


,video_name,seq,finger_x,finger_y,thumb_x,thumb_y
0,OC01_R_10s_lowfps.mp4,img000.png,527.435088,643.753157,568.125216,694.943964
1,OC01_R_10s_lowfps.mp4,img001.png,535.072768,643.354545,567.283288,691.011391
2,OC01_R_10s_lowfps.mp4,img002.png,533.811996,637.657582,567.946800,693.272491
3,OC01_R_10s_lowfps.mp4,img003.png,527.324229,636.657715,567.484718,691.878388
4,OC01_R_10s_lowfps.mp4,img004.png,528.169044,636.975038,567.431449,691.435793


In [7]:
## Now updating the labelled coordinates of the thumb and finger as per the cropped image
b_box = pd.read_csv("C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Boundingbox_dim_files/file_with_updated_centres_dups_removed.csv")
b_box.head()

labels_file_updt = pd.merge(labels_file, b_box, left_on='video_name', right_on='filename', how='inner').drop('filename', axis=1)
labels_file_updt.head()

,video_name,seq,finger_x,finger_y,thumb_x,thumb_y,new_xmin,new_ymin
0,OC01_R_10s_lowfps.mp4,img000.png,527.435088,643.753157,568.125216,694.943964,338,170
1,OC01_R_10s_lowfps.mp4,img001.png,535.072768,643.354545,567.283288,691.011391,338,170
2,OC01_R_10s_lowfps.mp4,img002.png,533.811996,637.657582,567.946800,693.272491,338,170
3,OC01_R_10s_lowfps.mp4,img003.png,527.324229,636.657715,567.484718,691.878388,338,170
4,OC01_R_10s_lowfps.mp4,img004.png,528.169044,636.975038,567.431449,691.435793,338,170


In [8]:
## Extracting this file for checking accuracy of hand detection model
labels_file_updt.to_csv(output_files + "combined_labels_orig_only_to_check_hand_detect_accuracy.csv")

In [135]:
# updating the coordinates as per cropped bounding box
labels_file_updt["new_finger_x"] = labels_file_updt["finger_x"] - labels_file_updt["new_xmin"] 
labels_file_updt["new_finger_y"] = labels_file_updt["finger_y"] - labels_file_updt["new_ymin"] 
labels_file_updt["new_thumb_x"] = labels_file_updt["thumb_x"] - labels_file_updt["new_xmin"] 
labels_file_updt["new_thumb_y"] = labels_file_updt["thumb_y"] - labels_file_updt["new_ymin"] 

#labels_file_updt.head()

labels_file_updt.iloc[:,[0,1,8,9,10,11]].to_csv(output_files + "combined_labels_v2_updated_xy.csv")
print("all done!!")

all done!!


In [2]:
print("note in v3, manually removed P21 as it had 85 frames")

note in v3, manually removed P21 as it had 85 frames


In [4]:
## delete below after using
# import cv2
# import pandas as pd
# import random
# import numpy as np
# xx = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/Train/' 

# image = cv2.imread(xx + "OC01_R_10s_lowfps.mp4@49.jpg", cv2.IMREAD_GRAYSCALE)


In [5]:
# old_size = image.shape[:2]
# desired_size = max(old_size)
# ratio = float(desired_size)/max(old_size)
# new_size = tuple([int(x*ratio) for x in old_size])
# new_size
# delta_w = desired_size - new_size[1]
# delta_h = desired_size - new_size[0]
# top, bottom = delta_h//2, delta_h-(delta_h//2)
# left, right = delta_w//2, delta_w-(delta_w//2)

# color = [0,0,0]
# image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,
#     value=color)

# print("adjust coordinates after this")

adjust coordinates after this
